# Crew AI

### Install Dependencies

In [233]:
%pip install crewai langchain_community langchain-groq python-dotenv 'crewai[tools]'



215339.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


### Imports

In [234]:
import os
import streamlit as st
from dotenv import load_dotenv 
load_dotenv()
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
# using openai
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Using Groq
model_name="gemma-7b-it"
# model_name="llama3-70b-8192"
# model_name="llama3-8b-8192"
os.environ["OPENAI_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = model_name
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"



# Groq
from langchain_groq import ChatGroq

# CrewAI
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, tool


### Init Groq

In [235]:
groq_llm = ChatGroq(
            temperature=0, 
            model_name="gemma-7b-it"
            # model_name="llama3-70b-8192"
            # model_name="llama3-8b-8192"
        )

### Tools

In [236]:
search_tool = SerperDevTool()


@tool("Spelling Checker")
def spelling_checker(text: str) -> str:
    """Useful when you finish a article and want to check for spelling mistakes"""
    
    return text + " Checked!!!"


### Create the Agents

In [237]:
# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Researcher',
  goal='Research specific topics on the web and provide summarized information',
  verbose=True,
  memory=False,
  backstory=(
    "Equipped with web search specialized in extracting and summarizing the most relevant information from a vast array "
    "of online resources. Using skills to stay updated on emerging trends and essential data."
  ),
  tools=[search_tool],
  allow_delegation=True,
  llm=groq_llm
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Create engaging and informative blog posts on assigned topics',
  verbose=True,
  memory=False,
  backstory=(
        "As a dedicated Blog Writer within a dynamic digital content team, you excel in transforming "
        "complex topics into readable, engaging blog posts that capture readers' interest and convey information "
        "effectively. Specializing in creating content that is not only informative but also captivates the audience, "
        "using a rich understanding of language and audience engagement without the need for external tools."
    ),
  tools=[search_tool, spelling_checker],
  allow_delegation=False,
  llm=groq_llm
)



### Create the Tasks

In [238]:
# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest {topic} trends.',
  tools=[search_tool],
  agent=researcher,
  async_execution=False,
  output_file='research-output.md'
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 6 paragraph article on {topic} advancements formatted as markdown.',
  agent=writer,
  tools=[search_tool, spelling_checker],
  async_execution=False,
  output_file='new-blog-post.md',
)
write_tweet = Task(
  description=(
   "Read the provided article about {topic} and use your summarization skills to craft a tweet that"
    " captures the article's essence in a compelling way. Add hashtags and mentions as needed."
  ),
  expected_output='A tweet-length summary of the article.',
  agent=writer,
  tools=[],
  async_execution=False,
  output_file='tweet.md',
)

### Create the Crew

In [239]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=False,
  cache=True,
  max_rpm=29, 
  verbose=2,
)

2024-05-03 14:28:41,264 - 140509684897600 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


### Initiate the work

In [240]:
result = crew.kickoff(inputs={'topic': 'Healthtech security'})
print("--------------------")
print("--------------------")
print(result)

 [DEBUG]: == Working Agent: Researcher
 [INFO]: == Starting Task: Identify the next big trend in Healthtech security.Focus on identifying pros and cons and the overall narrative.Your final report should clearly articulate the key points,its market opportunities, and potential risks.


> Entering new CrewAgentExecutor chain...


## Thought:

Action: Search the internet
Action Input: query="Healthtech security trends"## Thought:

Action: Search the internet
Action Input: {"query": "Healthtech security trends"} 

I encountered an error while trying to use the tool. This was the error: SerperDevTool._run() missing 1 required positional argument: 'search_query'.
 Tool Search the internet accepts these inputs: Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.

## Thought:

Action: Search the internet
Action Input: {"query": "Healthtech security trends"} 

I encountered an error while trying to use the tool. This was the error: SerperDevTool._run() missing 1 required positional argument: 'search_query'.
 Tool Search the internet accepts these inputs: Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.

## Thought:

Action: Search the internet
Action Input: {"query": "Healthtech secu